# Full forward pass & decode with Mobilenet + ImageNet

## 1.- Imports

In [ ]:
import numpy as np
import tensorflow_datasets as tfds
import tensorflow as tf
from matplotlib import pyplot as plt
from generate_mobilenet import Mobilenet
import pathlib
from tensorflow.python.keras import backend
from tensorflow.python.keras.utils import all_utils as keras_utils
import json
from mpl_toolkits.axes_grid1 import AxesGrid

In [ ]:
def plot_subplots(**kwargs):
    '''
    Function to plot several imagenes easily.
    kwargs:
    - dims: dimension of the subplots
    - imgs: list of images to plot
    '''
    with plt.style.context("dark_background"):
        fig = plt.figure(figsize=(6, 4))
        grid = AxesGrid(fig, 111,
                        nrows_ncols=kwargs['dims'],
                        axes_pad=0.05,
                        cbar_mode='single',
                        cbar_location='right',
                        cbar_pad=0.1
                        )
        for i, ax in enumerate(grid):
            ax.set_axis_off()
            im = ax.imshow(kwargs['imgs'][i], vmin=0, vmax=1)
        cbar = ax.cax.colorbar(im)
        cbar = grid.cbar_axes[0].colorbar(im)
        cbar.ax.set_yticks(np.arange(0, 1.1, 0.5))
        cbar.ax.set_yticklabels(['low', 'medium', 'high'])
        plt.show()
        plt.close(fig)

## 2.- Model instantiation

In [ ]:
path = pathlib.Path('.').parent.joinpath('debug', 'notebooks').resolve()
mobilenet = Mobilenet('MobileNet', path)
mobilenet.build(num_classes=500)

## 3.- ImageNet labels

In [ ]:
def get_imagenet_labels(last, num_classes):
    labels_path = tf.keras.utils.get_file(
        'ImageNetLabels.txt',
        'https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt')
    imagenet_labels = np.array(open(labels_path).read().splitlines())
    print('total # of labels: ', len(imagenet_labels))
    first_100 = imagenet_labels[-num_classes:]
    last_100 = imagenet_labels[:num_classes]

    imagenet_labels = last_100 if last else first_100
    print('After prunning: ', imagenet_labels.shape)
    print(f'''
    Labels in partition:
    \t- pizza: {'pizza' in imagenet_labels}
    \t- scuba diver: {'scuba diver' in imagenet_labels}
    \t- military uniform: {'military uniform' in imagenet_labels}
    \t- dog: {'dog' in imagenet_labels}
    \t- goldfish: {'goldfish' in imagenet_labels}
    \t- salamander: {'spotted salamander' in imagenet_labels}
    ''')
    print(imagenet_labels)
    return imagenet_labels
imagenet_labels = get_imagenet_labels(True, 500)


## 4.- Example image and preprocess
The two links provided are from TensorFlow and Wikipedia
![TensorFlow][https://storage.googleapis.com/download.tensorflow.org/example_images/grace_hopper.jpg]
![Wikipedia][https://upload.wikimedia.org/wikipedia/commons/thumb/9/97/Dog_for_Senior_Dog_Food_Diet_Wikipedia_Page.jpg/1200px-Dog_for_Senior_Dog_Food_Diet_Wikipedia_Page.jpg]

In [ ]:
def preprocess_url(url, name):
    f = tf.keras.utils.get_file(name, url)
    img = tf.keras.preprocessing.image.load_img(f, target_size=[159, 159])
    img = tf.keras.preprocessing.image.img_to_array(img)
    x = tf.keras.applications.mobilenet.preprocess_input(img)
    x = (x-np.min(x))/(np.max(x)-np.min(x))
    return x[tf.newaxis,...]

def download_example_imgs():
    goldfish_url = 'https://i.ytimg.com/vi/DSDUCxvb7R4/hqdefault.jpg'
    pizza_url = 'https://upload.wikimedia.org/wikipedia/commons/2/2d/Punch_Neapolitan_Pizza_Margherita.jpg'
    scuba_diver_url = 'https://fthmb.tqn.com/VHeLk32zZhv3Kp70RBinGvF_dmM=/3920x2160/filters:fill(auto,1)/GettyImages-597572085-584819855f9b5851e593fd0b.jpg'
    hopper_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/grace_hopper.jpg"
    salamander1_url = 'https://upload.wikimedia.org/wikipedia/commons/6/6a/Blue-spotted_salamander_(Ambystoma_laterale).jpg'
    dog_url = 'https://upload.wikimedia.org/wikipedia/commons/thumb/9/97/Dog_for_Senior_Dog_Food_Diet_Wikipedia_Page.jpg/1200px-Dog_for_Senior_Dog_Food_Diet_Wikipedia_Page.jpg'

    goldfish = preprocess_url(goldfish_url, 'fish')
    pizza = preprocess_url(pizza_url, 'pizza')
    scuba = preprocess_url(scuba_diver_url, 'scuba')
    hopper = preprocess_url(hopper_url, 'hopper')
    salamander = preprocess_url(salamander1_url, 'salamand')
    dog = preprocess_url(dog_url, 'dog')
    imgs = [goldfish, pizza, scuba, hopper, salamander, dog]
    return imgs

In [ ]:
imgs = download_example_imgs()
print('Range of values: ', np.max(imgs[0]), np.min(imgs[0]))
plot_subplots(imgs=[im[0] for im in imgs], dims=(2,3))

## 5.- Mobilenet output

In [ ]:
CLASS_INDEX = None
CLASS_INDEX_PATH = ('https://storage.googleapis.com/download.tensorflow.org/'
                    'data/imagenet_class_index.json')

def decode_predictions(preds, top=5, **kwargs):
    """Decodes the prediction of an ImageNet model.
    # Arguments
        preds: Numpy tensor encoding a batch of predictions.
        top: Integer, how many top-guesses to return.
    # Returns
        A list of lists of top class prediction tuples
        `(class_name, class_description, score)`.
        One list of tuples per sample in batch input.
    # Raises
        ValueError: In case of invalid shape of the `pred` array
            (must be 2D).
    """
    global CLASS_INDEX

    if len(preds.shape) != 2: #or preds.shape[1] != 1000:
        raise ValueError('`decode_predictions` expects '
                         'a batch of predictions '
                         '(i.e. a 2D array of shape (samples, 1000)). '
                         'Found array with shape: ' + str(preds.shape))
    if CLASS_INDEX is None:
        fpath = keras_utils.get_file(
            'imagenet_class_index.json',
            CLASS_INDEX_PATH,
            cache_subdir='models',
            file_hash='c2c37ea517e94d9795004a39431a14cb')
        with open(fpath) as f:
            CLASS_INDEX = json.load(f)
    results = []
    for pred in preds:
        top_indices = np.argsort(pred)[-top:][::-1]
        result = [tuple(CLASS_INDEX[str(i)]) + (pred[i],) for i in top_indices]
        result.sort(key=lambda x: x[2], reverse=True)
        results.append(result)
    return results

In [ ]:
output_goldfish = mobilenet.core_model(imgs[0])
print(output_goldfish.shape)
idx = np.argmax(output_goldfish)
print(idx)
imagenet_labels[idx+1]

In [ ]:
decoded = imagenet_labels[np.argsort(output_goldfish)[0,::-1][:5]+1]
print(decoded)

In [ ]:
decode_predictions(output_goldfish)

## 6.- Decoded output

In [ ]:
decoded = imagenet_labels[np.argsort(output_goldfish)[0,::-1][:5]+1]
print(decoded)

In [ ]:
decoded = imagenet_labels[np.argsort(output_pizza)[0,::-1][:5]+1]
print(decoded)